## В данном ноутбуке будет описан полный цикл тренировки нейронной сети на numpy

Шаг 1. Сначала скачаем данные для тренировки теста и валидации. Это данные с рукописными цифрами - MNIST.

In [ ]:
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/35c067adcc1ab364c8803830cdb34d0d50eea37e/week01_backprop/util.py -O util.py
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/35c067adcc1ab364c8803830cdb34d0d50eea37e/week01_backprop/mnist.py -O mnist.py
from __future__ import print_function
import numpy as np
np.random.seed(42)

Шаг 2. Опишем общий шаблон класса для каждого слоя нейросети. У каждого слоя нашей сетки будет всего 3 очень важных метода - это:

1. Метод init для инициализации параметров и весов сети
2. Метод forward, который принимает вход и выдает выход из нейросети
3. Метод backward который принимает на вход два параметра: вход в текущий слой нейросети и градиент функции потерь по выходу для того, чтобы прокинуть градиент от выхода на вход.

In [ ]:
class Layer:
    """
    Это абстрактный класс для блока нейросети. Каждый слой должен уметь делать две вещи:
    
    - Превращать входные данные в выходные:          output = layer.forward(input)
    
    - Прокидывать градиент через слой в обратную сторону:             grad_input = layer.backward(input, grad_output)
    
    
    Также некоторые слои будут иметь тренируемы параметры, которые должны обновляться в методе backward
    """
    def __init__(self):
        """Инициализация параметрами"""
        # В самом простом случае инициализировать нечего
        pass
    
    def forward(self, input):
        """
        Берет входные данные размера [batch, input_units] и возвращает [batch, output_units] 
        """
        # В самом простом случае просто возвращаем вход
        return input

    def backward(self, input, grad_output):
        """
        Пробрасывает градиент через слой.
        
        Для того, чтобы посчитать градиент по входу нейобходимо применить алгоритм обратного распространения ошибки:
        
        d loss / d x  = (d loss / d layer) * (d layer / d x)
        
        В данной формуле уже известно, что grad_output = (d loss / d layer) и необходимо посчитать только градиенты 
        текущего слоя по входу - (d layer / d x) и перемножить
        
        Если у слоя есть параметры, то необходимо их обновить с помощью d loss / d layer
        """
        # в простейше случае просто возвращаем grad_output и ничего не обновляем, но напишем это немного по-другому
        num_units = input.shape[1]
        
        d_layer_d_input = np.eye(num_units)
        
        return np.dot(grad_output, d_layer_d_input) # chain rule

Что будет дальше?

Далее напишем несколько блоков для нашей сетки:

1. Линейный полносвязный слой - $f(X)=X \cdot W + \vec{b}$
2. Слой активации ReLU
3. Функцию потерь crossentropy
4. Алгоритм обратного распространения ошибки - стохастический градиентный спуск

In [ ]:
class ReLU(Layer):
    def __init__(self):
        """Слой ReLU для нелинейности"""
        pass
    
    def forward(self, input):
        """Необходимо применить к каждому элементу input - [batch, input_units]"""
        # <your code. Try np.maximum>
        return output
    
    def backward(self, input, grad_output):
        """Compute gradient of loss w.r.t. ReLU input"""
        # <your code>
        # relu_grad = ...
        return grad_output * relu_grad        

In [ ]:
# some tests
from util import eval_numerical_gradient
x = np.linspace(-1,1,10*32).reshape([10,32])
l = ReLU()
grads = l.backward(x, np.ones([10,32])/(32*10))
numeric_grads = eval_numerical_gradient(lambda x: l.forward(x).mean(), x=x)
assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0),\
    "gradient returned by your layer does not match the numerically computed gradient"

Теперь сделаем что-нибудь немного более сложеное - линейные слой. В отличии от ReLU - линейный слой имеет обучаемые параметры внутри. Напомню форумулу: $f(X)=X \cdot W + \vec{b}$, где:

1. X - [batch, num_inputs]
2. W - [num_inputs, num_outputs]
3. b - вектор размерности num_outputs

In [ ]:
class Linear(Layer):
    def __init__(self, input_units, output_units, learning_rate=0.1):
        """
        f(x) = <x*W> + b
        """
        self.learning_rate = learning_rate
        
        # Инициализируем W небольшими числами из нормального распределения
        # Есть лучшие инициализации весов сеток - за ними сюда: http://bit.ly/2vTlmaJ
        self.weights = np.random.randn(input_units, output_units)*0.01
        self.biases = np.zeros(output_units)
        
    def forward(self,input):
        return #<your code here>
    
    def backward(self,input,grad_output):
        """
        Тут нужно продеферинцировать выход по входу для того, чтоб это вернуть на предыдущий слой
        Также нужно продеферинцировать выход по весам и биасам для того, чтобы сделать град спуск по обучаемым параметрам
        """
        # вычислить d f / d x = d f / d dense * d dense / d x
        grad_input = #<your code here>
        
        # вычислить градиенты по параметрам
        grad_weights = #<your code here>
        grad_biases = #<your code here>
        
        assert grad_weights.shape == self.weights.shape and grad_biases.shape == self.biases.shape
        # Here we perform a stochastic gradient descent step. 
        # Later on, you can try replacing that with something better.
        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases
        
        return grad_input

In [ ]:
l = Linear(128, 150)

assert -0.05 < l.weights.mean() < 0.05 and 1e-3 < l.weights.std() < 1e-1,\
    "The initial weights must have zero mean and small variance. "\
    "If you know what you're doing, remove this assertion."
assert -0.05 < l.biases.mean() < 0.05, "Biases must be zero mean. Ignore if you have a reason to do otherwise."

# To test the outputs, we explicitly set weights with fixed values. DO NOT DO THAT IN ACTUAL NETWORK!
l = Linear(3,4)

x = np.linspace(-1,1,2*3).reshape([2,3])
l.weights = np.linspace(-1,1,3*4).reshape([3,4])
l.biases = np.linspace(-1,1,4)

assert np.allclose(l.forward(x),np.array([[ 0.07272727,  0.41212121,  0.75151515,  1.09090909],
                                          [-0.90909091,  0.08484848,  1.07878788,  2.07272727]]))
print("Well done!")

### The loss function

Поскольку мы хотим предсказывать вероятности было бы логично сделать softmax, а потом уже функцию потерь. В данном случае будем реализовывать crossentropy loss.

Обычный cross entropy loss: $$ loss = -\sum y_i log \hat{y_i}$$

В данном случае получится:

$$ loss = - log \space {e^{a_{correct}} \over {\underset i \sum e^{a_i} } } $$

Лучше переписать вот так:

$$ loss = - a_{correct} + log {\underset i \sum e^{a_i} } $$

Последнее выражение лучше всего, поскольку:
* Легче считать производные
* Обычно быстрее считается

In [ ]:
def softmax_crossentropy_with_logits(logits,target):
    """logits[batch,n_classes] target - [batch]. Подсчет лосса"""
    logits_for_answers = logits[np.arange(len(logits)),target]
    
    xentropy = #<your code here>
    
    return xentropy

def grad_softmax_crossentropy_with_logits(logits,target):
    """logits[batch,n_classes] target - [batch]. Градиент по лоссу"""
    ones_for_answers = np.zeros_like(logits)
    
    #<your code here>
    
    return grad

In [ ]:
logits = np.linspace(-1,1,500).reshape([50,10])
answers = np.arange(50)%10

softmax_crossentropy_with_logits(logits,answers)
grads = grad_softmax_crossentropy_with_logits(logits,answers)
numeric_grads = eval_numerical_gradient(lambda l: softmax_crossentropy_with_logits(l,answers).mean(),logits)

assert np.allclose(numeric_grads,grads,rtol=1e-3,atol=0), "The reference implementation has just failed. Someone has just changed the rules of math."

Шаг 3. Теперь давайте скомбинируем все слои вместе и построим нейронку для классификации рукописных цифр.

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline

from mnist import load_dataset
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(flatten=True)

plt.figure(figsize=[6,6])
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.title("Label: %i"%y_train[i])
    plt.imshow(X_train[i].reshape([28,28]),cmap='gray');

Теперь определим нашу нейронную сеть

In [ ]:
net = []
net.append(Linear(X_train.shape[1], 100))
net.append(ReLU())
net.append(Linear(100, 200))
net.append(ReLU())
net.append(Linear(200, 10))

Напишем foward

In [ ]:
def forward(net, X):
    """
    Подсчет активаций для каждого слоя нейросети. Последняя активация - это логиты
    """
    activations = []
    input = X
    
    #<your code here>
        
    assert len(activations) == len(net)

    return activations

def predict(net, X):
    """
    предсказание меток
    """
    #<your code here>

Напишем тренировку с backprop

In [ ]:
def train(net,X,y):
    # Активации слоев
    layer_activations = forward(net,X)
    layer_inputs = [X] + layer_activations[:-1]
    logits = layer_activations[-1]
    
    # Вычисляем лосс и его градиент
    loss = softmax_crossentropy_with_logits(logits,y)
    loss_grad = grad_softmax_crossentropy_with_logits(logits,y)    
    # алгоритм backprop
    
    #<your code here>
    
    return np.mean(loss)

Теперь все готово! Можно запускать...

In [ ]:
from tqdm import trange
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
from IPython.display import clear_output
train_log = []
val_log = []

In [ ]:
for epoch in range(25):

    for x_batch,y_batch in iterate_minibatches(X_train, y_train, batchsize=32, shuffle=True):
        train(net, x_batch, y_batch)
    
    train_log.append(np.mean(predict(net, X_train) == y_train))
    val_log.append(np.mean(predict(net, X_val) == y_val))
    
    clear_output()
    print("Epoch",epoch + 1)
    print("Train accuracy:",train_log[-1])
    print("Val accuracy:",val_log[-1])
    plt.plot(train_log,label='train accuracy')
    plt.plot(val_log,label='val accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.show()